In [1]:
import argparse
import os
import sys
import json
import shutil
import pathlib
import random
from itertools import islice
import time
from copy import deepcopy
import math
from pprint import pprint
from qj_global import qj
import logging

try:
    from comet_ml import Experiment
    COMET_AVAIL = True
except:
    COMET_AVAIL = False

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

from ignite.engine import Engine, Events
from ignite.handlers import Checkpoint, DiskSaver, Timer, TerminateOnNan, TimeLimit
from ignite.metrics import RunningAverage
from ignite.contrib.metrics import GpuInfo
from ignite.utils import setup_logger

# adding the folder containing the folder `disentanglement_via_mechanism_sparsity` to sys.path
# sys.path.insert(0, str(pathlib.Path(__file__).parent.parent))
sys.path.insert(0, str('/Users/vitoria/Documents/GitHub/'))
from disentanglement_via_mechanism_sparsity.universal_logger.logger import UniversalLogger
from disentanglement_via_mechanism_sparsity.metrics import MyMetrics, linear_regression_metric, mean_corr_coef, edge_errors
from disentanglement_via_mechanism_sparsity.plot import plot_01_matrix
from disentanglement_via_mechanism_sparsity.data.synthetic import get_ToyManifoldDatasets
from disentanglement_via_mechanism_sparsity.model.ilcm_vae import ILCM_VAE
from disentanglement_via_mechanism_sparsity.model.latent_models_vae import FCGaussianLatentModel


In [2]:
manifold = "nn"
transition_model = "action_sparsity_trivial"
# transition_model = "action_sparsity_non_trivial"

In [3]:
datasets = get_ToyManifoldDatasets(manifold, transition_model, split=(0.8, 0.1, 0.1),
                                       z_dim=10, x_dim=20, num_samples=int(1e6),
                                       no_norm=True, discrete=True)

In [4]:
train_loader = data.DataLoader(datasets[4], batch_size=500, shuffle=False, drop_last=True)

In [5]:
train_loader

In [6]:
train_batch = next(iter(train_loader))

In [22]:
train_batch[7].shape

torch.Size([500, 1, 5, 20])

In [18]:
train_batch[5].shape

torch.Size([500, 1, 5, 10])

In [15]:
len(datasets[4].__getitem__(5))

8

In [9]:
datasets[4].disc_x.shape

torch.Size([800000, 5, 20])

In [10]:
datasets[4].p_x.shape

torch.Size([800000, 5, 20])

In [11]:
datasets[4].disc_z.shape

torch.Size([800000, 5, 10])

In [12]:
datasets[4].p_z.shape

torch.Size([800000, 5, 10])